## SHA

SHA stands for "Secure Hash Algorithm". SHA is a collection of cryptographic hash functions standardized by the United States National Institute of Standards and Technology (NIST.) Periodically, they hold design competitions to choose a new hash function design. Currently, the SHA-2 family is the most widely adopted. SHA-1 is no longer considered secure, and SHA-3 is somewhat new. Usually, when someone mentions SHA, they are referring to a hash from the SHA-2 family.

Family? Yes. There are a number of different SHA-2 configurations. There are several options for the number of bits in the digest, 256 and 512 being the most common. When someone refers to SHA-256 or SHA-512, they mean SHA2 with a 256 or 512 bit digest.

We can compute SHA using the built-in Python libraries. Unfortunately, it's not as simple as you might imagine. Here's the code.

In [ ]:
import hashlib

inputString="HELLO WORLD!".encode('ASCII')
print(hashlib.sha256(inputString).hexdigest())

There are a couple things that might look strange to you here. If you're comfortable using the code and ignoring those quirks, no problem. I'll explain them in the paragraph below if you're curious.

The first weird thing is that we need to tell Python how to encode our string. There are many ways to represent a string in bytes (e.g. ASCII or UTF-8 or UTF-32.) We must be explicit about this to invoke hashlib's sha256 function, because the hash digest depends on the precise string encoding. The second strange thing you might have noticed is that we need to invoke the "hexdigest" function on the return value. The hashlib function returns a fancy datatype that we could use to do many fancy things. But we don't want to do fancy things. We just want the hash digest. We can get it by asking that fancy datatype to call the hexdigest function.

## The Avalanche Effect

Previously, we explored three desirable properties of cryptographic hashes (preimage, second-preimage, and collision resistance) by demonstrating that badHash did not satisfy them. Let's explore another property and demonstrate that SHA-256 **does** satisfy it.

Ideally, we want every single individual bit of the input string to have some influence on every bit of the hash digest. Intuitively, this would mean every single bit of the output depends on every single bit of the input. We've smeared whatever "information" was present in the input string all across the output, and there's nothing we can learn about the input from those output bits.

Another way to think about the avalanche effect is that it's like the "butterfly effect". If a small changes in the input can have a big impact on the output, predicting the output is very difficult.

Yet another way to think about it is that even if we know all of the bits of the input except **one**, each bit in the output looks like a coin toss. One unknown input bit is enough to make the output digest seem random.

Let's demonstrate that SHA-256 satisfies the avalanche effect. First, I will write a function for you that counts the number of bits different between two integers x and y. You don't need to understand how it works. You can just invoke this function when you need it.

In [ ]:
def countBits(x,y):
    return bin(x^y).count('1')

countBits(0xF,0xA)

Now we just need two strings that differ by one bit. Fortunately, ASCII uppercase and lowercase encodings always differ by exactly one bit.

In [ ]:
print(countBits(ord('A'),ord('a')))
print(countBits(ord('B'),ord('b')))
print(countBits(ord('C'),ord('c')))

So let's find the SHA-256 digests of two strings, convert them integers, and then count the number of bits they differ by.

In [ ]:
inputA="It comes back to you in the end. Boomerang. Respect it.".encode('ASCII')
inputB="It comes back to you in the end. Boomerang. Respect it.".encode('ASCII')
digestA=hashlib.sha256(inputA).hexdigest()
digestB=hashlib.sha256(inputB).hexdigest()
intA=int(digestA,16)
intB=int(digestB,16)
countBits(intA,intB)

Uh oh. Looks like I forgot to make the strings different. Try making the strings differ by one bit by changing the capitalization of one of the letters in one of the strings. How many bits change in the output? If you do this a few times, you should see an average of about 128 or half of 256.

## Attacking SHA

There are clever ways to attack the inner workings of SHA-2 hashes, but they're still largely theoretical. In this section, I want to remind you that even "military grade" cryptography can't protect us if we don't use it properly. Here's the hash digest for a string: b7a56873cd771f2c446d369b649430b65a756ba278ff97ec81bb6f55b2e73569.

We shouldn't be able to guess the input string, right? Well, what if I told you the input string was just 2 characters long, and both characters were digits? There are only 100 possible inputs. We can just guess all the possible inputs to find the one that matches the input hash.

Here is code to guess every possible input string. What input string produced the hash digest above?

In [ ]:
for i in '0123456789':
    for j in '0123456789':
        x=(i+j).encode('ASCII')
        print(i+j,hashlib.sha256(x).hexdigest())

Remember, this attack was not successful because of a vulnerability in SHA-256. The flaw was in the **input**, not the hash function. Our input string was too easy to guess. Does this remind you of anything you know about passwords? You've probably heard that you should choose a password that's long and difficult to guess. Do you think that's related to the attack we just demonstrated? Yes! We'll talk more about it later, but **password cracking** is the process of guessing people's passwords from their hash digests.

## Random Number Generation

You're going to see that random numbers are very important for cryptography. We need lots of truly unpredictable bits to build secure systems. For example, you know you want a secure password. Nobody should be able to guess it. How would an attacker be able to guess your password? Something about it needs to be predictable. Predictablility is a weakness. A secure password should be unpredictable. Nothing is more unpredictable than randomness. If you want a ridiculously secure password, roll a dice 30 times and memorize the result.

Unfortunately, computers are deterministic. By their very design, they cannot produce anything truly random. They always do exactly what they're told and nothing more.

That's not to say we can't find sources of randomness inside a computer. For example, the precise location of the mouse and the exact temperature of the CPU contain some amount of true randomness. We call this random noise **entropy**. The operating system measures and stores these sources of entropy. We call this collection of truely random values the **entropy pool**.

Python can retrieve some bytes of random data for us from the operating system's entropy pool. To do this, we import the **os** library and use its urandom function. The code below gets 256 bytes of random data from the operating system and converts it into a hexadecimal string. Every time you run it, you should get a different result.

In [ ]:
import os

os.urandom(256).hex()

So there we go! Randomness! Problem solved, right? Well... remember how I said we need lots of unpredictable numbers? Turns out we need **LOTS** of unpredictable numbers. The entropy pool just isn't big enough for all of our modern cryptographic needs. Fortunately, we can generate **pseudorandom numbers**.

Pseudorandom numbers are algorithmically generated. They're not truly random, but they're still unpredictable. If I gave the smartest cryptographers in the world a sequence of truly random numbers and a sequence of pseudorandom numbers, they shouldn't be able to tell the difference.

We want a program whose output is indistinguishable from randomness. In other words, we can't predict the output. In other **other** words, the output tells us nothing about the inputs we used to compute the result. Sound familiar? It should, because that's what a cryptographic hash does! 

If we put a string into SHA-256, we get out 256 bytes that are unpredictable to anyone who doesn't know the input string. We have 256 pseudorandom bytes! That's not enough. We need more!

What if we made our input string a number? For example, we hash "1". To get more pseudorandom bytes, why not hash "2", then "3", then "4". We could go on as long as we want and get as many pseudorandom bytes as we like! Here is Python code that uses a loop to count from 0 to 19. Inside the loop, it converts the integer into a string and computes its SHA256 digest. We concatenate all of the digests together in a string called randomBytes and print them out. $256 \times 20=5,120$ pseudorandom bytes!

In [ ]:
import hashlib
randomBytes=""
for i in range(0,20):
    inputString=str(i).encode('ASCII')
    randomBytes+=hashlib.sha256(inputString).hexdigest()
print(randomBytes)

One problem. We already know that cryptographic hashes can't protect us if the input is easy to guess. Our counter started at 0. If the attacker can guess that, they can get the same pseudorandom stream.

We call this starting value the **seed** of our pseudorandom number generator. We want it to be unpredictable. How about we use a truly random number from the entropy pool? Then the seed will be literally unpredictable. That's exactly what the code below does. If we wanted, we could use this method to easily produce billions of bytes (gigabytes) of cryptographically secure pseudorandom numbers.

In [ ]:
import hashlib
randomBytes=""
seed=int(os.urandom(256).hex(),16)
for i in range(seed,seed+20):
    inputString=str(i).encode('ASCII')
    randomBytes+=hashlib.sha256(inputString).hexdigest()
print(randomBytes)

## HMACs

Cryptographic hash functions are the workhorses of modern cryptography. They have many diverse applications, but the main reason we're studying them in this class is that they can be used to guarantee **integrity**. When Alice sends Bob a message, he wants to know that Mallory hasn't tampered with it.

Let's say Alice wants to send a message M to Bob. She can use a cryptographic hash function (e.g. SHA-256) to compute a message digest $H(M)=V$.

In [ ]:
import hashlib
M="MISSION COMPROMISED. ABORT."
V=hashlib.sha256(M.encode('ASCII')).hexdigest()
print(M,V)

If she can deliver the message digest V to Bob on a secure channel, Bob can check to make sure he gets the same result when he computes $H(M)$ on the message he received. If they match, Mallory didn't tamper with the message.

In [ ]:
M="MISSION COMPROMISED. ABORT."
V="b8f729784948ff3bc830069d36e970d86b3b19dc4cd73105c712466c2b7b70f3"
hashlib.sha256(M.encode('ASCII')).hexdigest()

If they don't match, Bob knows Mallory tampered with the message.

In [ ]:
M="YOU ARE GO FOR LAUNCH."
V="b8f729784948ff3bc830069d36e970d86b3b19dc4cd73105c712466c2b7b70f3"
hashlib.sha256(M.encode('ASCII')).hexdigest()

Sure, this is a form of integrity guerantee, but this is a little bit silly. We're assuming Mallory can't tamper with the message digest V. If she does, she could just change V to match her tampered message. If Mallory can tamper with V, Bob can no longer be certain that a match means Alice sent the message.

Why assume a weakness on the part of our attacker? We want to be able to communicate securly even if we know the communication channel is compromised. Fortunately, there is a way to improve on this.

First, Alice and Bob need to agree on a secret key. This key is known only to them. In the future, we'll talk about how they can exchange keys securely. For now, we can just imagine they meet in person under cover of darkness. Let's generate a secure 256-bit random key using the method we learned above.

In [ ]:
import os, hashlib
hmacKey=hashlib.sha256(os.urandom(256).hex().encode('ASCII')).hexdigest()
print(hmacKey)

Now for the clever trick. We know that Mallory can't learn anything about the input to a hash function from its output. What if Alice puts her message **and the key** into the hash function to produce the message digest V? What if she sends this to Bob?

In [ ]:
M="MISSION COMPROMISED. ABORT."
V=hashlib.sha256((hmacKey+M).encode('ASCII')).hexdigest()
print(M,V)

Bob knows the secret key. He can repeat this process to arrive at the same digest.

In [ ]:
hashlib.sha256((hmacKey+M).encode('ASCII')).hexdigest()

Why is this trick clever? Only a person who knows the key can produce that hash! Mallory doesn't know the key. She can't forge a message digest V for her messages without it! We have message integrity.

## Technical Caveat

Don't worry if this part goes over your head. I'm including it for the sake of completeness and accuracy.

The HMAC protocol I outlined above is insecure in practice, because of the way SHA-256 is constructed. Merkle–Damgard hashes have a known weakness when the attacker can concatinate to the end of messages. This is called a **length-extension attack**. To protect against this weakness, we need to modify our HMAC protocol. This new HMAC protocol uses SHA-256 twice. We can use a trick with XOR to produce **two** keys from our original key. We compute exactly what I described above with the first key $SHA(key_1+M)$ to get a digest $V_1$. Then we **repeat the process** with the second key $SHA(key_2,V_1)$ to arrive at the final message digest.

In practice, the secure version of an SHA-256 HMAC looks like something like this:

In [ ]:
c1=0x3636363636363636363636363636363636363636363636363636363636363636
c2=0x5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c5c

key1=hex(int(hmacKey,16)^c1)
key2=hex(int(hmacKey,16)^c2)

hashlib.sha256((key1+hashlib.sha256((key2+M).encode('ASCII')).hexdigest()).encode('ASCII')).hexdigest()

## Exercises
Below, you'll find the exercises for this notebook. Don't forget to take breaks and ask for help if you need them!

1) Compute the SHA-256 hash of the string "I ain't been tired but twice in my life, and both times is now." encoded in ASCII.

In [2]:
inputString="I ain't been tired but twice in my life, and both times is now."

2) Based on our previous discussion of the birthday paradox, roughly how many guesses would you need to find a pair of random input strings whose SHA-256 digests match?

3) How many bits do the SHA-256 digests of these two strings differ by? Use ASCII encoding.

In [3]:
inputA="I'm great at boats.".encode('ASCII')
inputB="i'm great at boats.".encode('ASCII')

4) What input string produced this SHA-256 hash? The input was a single lower-case letter.

In [5]:
digest='2d711642b726b04401627ca9fbac32f5c8530fb1903cc4db02258717921a4881'

5) Below are two messages and our HMAC key. One is real, the other is from Mallory. My digest was computed using the secure method outlined above. Which message did I send?

In [11]:
hmacKey=0x061213633fd574e20d5ebaa962c1c261654c883da159d00547e8633a0ac55715
digest=0xfae6c48ee44b49d33b91bafaa8ae7e8a706ccc3eabcd1bb4bb54d1ef97b907b1

message1="Okay. This looks bad."
message2="Aw, coffee, no."